In [1]:
import pandas as pd
import numpy as np
import cvxpy as cp


In [25]:
df = pd.read_csv("aoe4_data.csv")
print(list(df.columns))
df = df.dropna()


[' ', 'Unnamed: 1', 'Unnamed: 2', 'Resource Cost', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Time', 'Pop', 'Move', 'HP', 'HP per\nResource', 'Armor', 'Unnamed: 13', 'Attack', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Bonus', 'Note']


In [9]:

df.rename(columns={"Resource Cost": "Food", "Unnamed: 4": "Wood","Unnamed: 5": "Gold",}, inplace=True)
df.rename(columns={"Unnamed: 15" :"Spd","Unnamed: 16": "DPS","Unnamed: 17": "Rng"},inplace =True)
df.drop(columns=["Unnamed: 1", "Unnamed: 2","Bonus","Note","Unnamed: 19","Unnamed: 18","Unnamed: 13","Unnamed: 6","Move", "DPS", "HP per\nResource"], inplace=True)
df = df.iloc[:124]
df = df.dropna(subset=['Food', 'Gold', 'Wood'])

In [ ]:
df.head()
df = df.dropna()
# 100 wood is 130 gold
# 100 food is 130 gold

In [11]:
df.Food *= 1.3
df.Wood *= 1.3
df.Gold = df.Food + df.Wood + df.Gold
df.drop(columns=["Food", "Wood"], inplace=True)
df.head()

,,Gold,Time,Pop,HP,Armor,Attack,Spd,Rng
2,Spearman,104.0,15.0,1.0,70.0,0,6,1.75,0.29
19,Longbowman,117.0,15.0,1.0,70.0,0,6,1.5,7.00
29,Arbalétrier,144.0,22.0,1.0,80.0,1,12,2,5.00
34,Scout,91.0,20.0,1.0,110.0,0,2,2,0.29
40,Camel Rider,294.0,35.0,1.0,270.0,0,14,1,0.42


In [12]:
# Time, HP, Pop, Armor, Attack, Speed and Range would be our constraints
df
print(len(df.index))


25


In [13]:
c = np.array(df['Gold'])
A = np.array(df[['Time','HP','Armor','Attack', "Spd", "Rng", "Pop"]]).T
df["Armor"] = df["Armor"].astype(float)
df["Attack"] = df["Attack"].astype(float)
df["Spd"] = df["Spd"].astype(float)
df["Rng"] = df["Rng"].astype(float)
b = np.array([1026,5000,20,1000,100,130,200]) # These are our requirements

In [14]:
X = cp.Variable(df.shape[0])
obj = cp.Minimize(c @ X)
constraint_0 = [A@X>=b]
constraint_1 = [X>=0]
prob = cp.Problem(obj,constraint_0+constraint_1)
prob.solve()

np.float64(13949.999992312105)

In [26]:
for i in range(df.shape[0]):
    print(df.iloc[i][' '],abs(X.value.round(3)[i]))

Spearman 0.0
Longbowman 0.0
Arbalétrier 20.0
Scout 0.0
Camel Rider 0.0
Fire Lancer 0.0
War Elephant 0.0
Camel Archer 0.0
Springald 0.0
Mangonel 0.0
Bombard 0.0
Cannon 0.0
Culverin 0.0
Galley 0.0
Junk 45.0
Light Junk 0.0
Dhow 0.0
Lodya Attack Ship 0.0
Demolition Ship 0.0
Explosive Junk 0.0
Explosive Dhow 0.0
Lodya Demolition Ship 0.0
Carrack 0.0
Baochuan 0.0
Xebec 0.0
